In [220]:
import numpy as np
import pandas as pd
from sklearn import neighbors, metrics
from sklearn.model_selection import KFold

### 1) Ré-implémentation d'une fonction de validation croisée
La fonction prend en entrée :
- le jeu d'entraînement,  
- les hyper-paramètres à tester,
- le nombre de folds pour la validation croisée

La fonction retourne :
- les résultats pour chaque valeur de l'hyperparamètre (mean, std)
- les valeurs testées pour l'hyper-paramètre
- la meilleure valeur pour l'hyper-paramètre 
- les sets train/test des folds pour pouvoir réutiliser les mêmes sets pour comparaison avec la fonction SearchGridCV
- le modèle entraîné avec le meilleur hyper-paramètre


In [221]:
def validation_croisee(X,y,parameters, n_folds) :   
    
    # constitutons les folds : sets de données train/test
    kf = KFold(n_splits = n_folds)            
    
    best_score = 0
    best_params = 0
    mean_values = []
    std_values = []
    
    # pour chaque valeur d'hyper-paramètre, calcul de la moyenne du score obtenu sur chaque fold
    for num_neighbors in parameters :
        scores = []
        
        # pour chaque set de données (fold), entraînement du modèle sur les données train et score sur les données test
        
        for train_index, test_index in kf.split(X) :
            X_CV_train, X_CV_test = X[train_index], X[test_index]
            y_CV_train, y_CV_test = y[train_index], y[test_index]    

            knn = neighbors.KNeighborsClassifier(n_neighbors=num_neighbors)
            knn.fit(X_CV_train, y_CV_train)
            score = knn.score(X_CV_test, y_CV_test)
            scores.append(score)
            
        # moyenne des scores pour num_neighbors fixé
        mean = np.mean(scores)
        mean_values.append(mean)
        std = np.std(scores)
        std_values.append(std)
        
        # mémorisation du meilleuir score et de l'hyper-paramètre associé
        if mean>best_score :
            best_score=mean
            best_params_ = num_neighbors
            best_knn = knn
        
    #on retourne les sets de données pour pouvoir les utiliser pour une comparaison avec searchGridCV sur des données identiques
    return({'mean_test_score' : mean_values, 'std_test_score' : std_values, 'params' : parameters, 'best_params_' :best_params_ , 'folds' : kf, 'trained_knn' : best_knn })

### 2) comparaison des résultats d'évaluation avec la fonction de validation croisée ré-implémentée et la fonction SearchGridCV

### Chargement des données

In [222]:
data = pd.read_csv('winequality-white.csv', sep = ";")
data.head()

X = data.as_matrix(data.columns[:-1])
y = data.as_matrix([data.columns[-1]])
y = y.flatten()

### Séparation en données d'entraînement et données de test

In [223]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = \
	model_selection.train_test_split(X, y,
                                	test_size=0.3 # 30% des données dans le jeu de test
                                	)

### Standardisation des données

In [224]:
from sklearn import preprocessing
std_scale = preprocessing.StandardScaler().fit(X_train)
X_train_std = std_scale.transform(X_train)
X_test_std = std_scale.transform(X_test)

### comparaison des résultats 

In [225]:
parameters = [3, 5, 7, 9, 11, 13, 15]
n_folds = 5

# lance la fonction de cross validation sur les données d'entraînement
# retourne kf = les sets de données des folds pour pouvoir réutiliser les mêmes avec la fonction GrisSearcgCV, pour comparaison sur de sets identiques

results_validation_croisee = validation_croisee(X_train_std, y_train, parameters, n_folds)
    
# Afficher le(s) hyperparamètre(s) optimaux
print("nouvelle implémentation de la fonction de cross validation")
print("===========================================================")

# Afficher le(s) hyperparamètre(s) optimaux
print ("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:",)
print (results_validation_croisee['best_params_'])

# Afficher les performances correspondantes
print ("Résultats de la validation croisée :")
for mean, std, params in zip(results_validation_croisee['mean_test_score'], # score moyen
results_validation_croisee['std_test_score'], # écart-type du score
results_validation_croisee['params'] # valeur de l'hyperparamètre
):
    print ("\t%s = %0.3f (+/-%0.03f) for %r" % (score, # critère utilisé
    mean, # score moyen
    std * 2, # barre d'erreur
    params # hyperparamètre
    ))
print('\n\n')  

nouvelle implémentation de la fonction de cross validation
Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:
7
Résultats de la validation croisée :
	accuracy = 0.518 (+/-0.054) for 3
	accuracy = 0.536 (+/-0.040) for 5
	accuracy = 0.543 (+/-0.037) for 7
	accuracy = 0.539 (+/-0.043) for 9
	accuracy = 0.538 (+/-0.036) for 11
	accuracy = 0.537 (+/-0.035) for 13
	accuracy = 0.536 (+/-0.029) for 15





In [226]:
# Fixer les valeurs des hyperparamètres à tester
param_grid = {'n_neighbors':parameters}

# Choisir un score à optimiser, ici l'accuracy (proportion de prédictions correctes)
score = 'accuracy'

cv = results_validation_croisee['folds']

# Créer un classifieur kNN avec recherche d'hyperparamètre par validation croisée
clf = model_selection.GridSearchCV(neighbors.KNeighborsClassifier(), # un classifieur kNN
param_grid, # hyperparamètres à tester
cv=kf, # nombre de folds de validation croisée
scoring=score # score à optimiser
)

# Optimiser ce classifieur sur le jeu d'entraînement
# clf.fit(X_train, y_train)
clf.fit(X_train_std, y_train)

print("implémentation de la fonction SearchGridCV")
print("==========================================")
    
# Afficher le(s) hyperparamètre(s) optimaux
print ("Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:",)
print (clf.best_params_)

# Afficher les performances correspondantes
print ("Résultats de la validation croisée :")
for mean, std, params in zip(clf.cv_results_['mean_test_score'], # score moyen
clf.cv_results_['std_test_score'], # écart-type du score
clf.cv_results_['params'] # valeur de l'hyperparamètre
):
    print ("\t%s = %0.3f (+/-%0.03f) for %r" % (score, # critère utilisé
    mean, # score moyen
    std * 2, # barre d'erreur
    params # hyperparamètre
    ))
print('\n\n')   


implémentation de la fonction SearchGridCV
Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:
{'n_neighbors': 7}
Résultats de la validation croisée :
	accuracy = 0.518 (+/-0.054) for {'n_neighbors': 3}
	accuracy = 0.536 (+/-0.040) for {'n_neighbors': 5}
	accuracy = 0.543 (+/-0.037) for {'n_neighbors': 7}
	accuracy = 0.539 (+/-0.043) for {'n_neighbors': 9}
	accuracy = 0.538 (+/-0.036) for {'n_neighbors': 11}
	accuracy = 0.537 (+/-0.035) for {'n_neighbors': 13}
	accuracy = 0.536 (+/-0.029) for {'n_neighbors': 15}





### Les résultats sont bien identiques

### 3) Application et évaluation du K-nn avec la fonction de validation croisé ré-implémentée

In [230]:
parameters = [3, 5, 7, 9, 11, 13, 15]
n_folds = 5

knn = validation_croisee(X_train_std, y_train, parameters, n_folds)['trained_knn']

y_pred = knn.predict(X_test_std)
print ("\nPerformance du modèle optimisé par la ré-implémentation d'une fonction de validation croisée Sur le jeu de test : %0.3f" % metrics.accuracy_score(y_test, y_pred))


Performance du modèle optimisé par la ré-implémentation d'une fonction de validation croisée Sur le jeu de test : 0.544
